In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Extract python codes from a file

In [2]:
import os

file_dir = '../../gpt-discord-bot/src/'
file_list = os.listdir(file_dir); file_list

['base.py',
 'completion.py',
 'config.yaml',
 'constants.py',
 'main.py',
 'moderation.py',
 'utils.py']

In [3]:
fname = os.path.join(file_dir, file_list[0]); fname
f = open(fname, "r")
code = f.read()
code

'../../gpt-discord-bot/src/base.py'

'from dataclasses import dataclass\nfrom typing import Optional, List\n\nSEPARATOR_TOKEN = "<|endoftext|>"\n\n\n@dataclass(frozen=True)\nclass Message:\n    user: str\n    text: Optional[str] = None\n\n    def render(self):\n        result = self.user + ":"\n        if self.text is not None:\n            result += " " + self.text\n        return result\n\n\n@dataclass\nclass Conversation:\n    messages: List[Message]\n\n    def prepend(self, message: Message):\n        self.messages.insert(0, message)\n        return self\n\n    def render(self):\n        return f"\\n{SEPARATOR_TOKEN}".join(\n            [message.render() for message in self.messages]\n        )\n\n\n@dataclass(frozen=True)\nclass Config:\n    name: str\n    instructions: str\n    example_conversations: List[Conversation]\n\n\n@dataclass(frozen=True)\nclass Prompt:\n    header: Message\n    examples: List[Conversation]\n    convo: Conversation\n\n    def render(self):\n        return f"\\n{SEPARATOR_TOKEN}".join(\n    

### Add prompt to code to extract `class` and `def`

In [4]:
prompt_postfix = """ 
###
Here's what the above class is doing:
1.
"""

In [6]:
prompt = code +  prompt_postfix
print(prompt)

from dataclasses import dataclass
from typing import Optional, List

SEPARATOR_TOKEN = "<|endoftext|>"


@dataclass(frozen=True)
class Message:
    user: str
    text: Optional[str] = None

    def render(self):
        result = self.user + ":"
        if self.text is not None:
            result += " " + self.text
        return result


@dataclass
class Conversation:
    messages: List[Message]

    def prepend(self, message: Message):
        self.messages.insert(0, message)
        return self

    def render(self):
        return f"\n{SEPARATOR_TOKEN}".join(
            [message.render() for message in self.messages]
        )


@dataclass(frozen=True)
class Config:
    name: str
    instructions: str
    example_conversations: List[Conversation]


@dataclass(frozen=True)
class Prompt:
    header: Message
    examples: List[Conversation]
    convo: Conversation

    def render(self):
        return f"\n{SEPARATOR_TOKEN}".join(
            [self.header.render()]
            + [Mess

### Request a response from API

In [7]:
#Note: The openai-python library support for Azure OpenAI is in preview.
import os
import openai
from dotenv import load_dotenv

load_dotenv()

openai.api_type = "azure"
openai.api_base = "https://tutorial-openai-01-2023.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

True

In [20]:
import sys
sys.path.insert(0, '../src')
from utils import prompt_api, print_response


In [21]:
engine="text-davinci-003"  # note that text-davinci-x responds better than code-davinci-x
prompt=prompt
temperature=0 # [0, 1]
max_tokens=300
top_p=1
frequency_penalty=0 # [0, 2]
presence_penalty=0 # [0, 2]
best_of=1
stop=["###"]

response = prompt_api(engine, prompt, temperature, max_tokens, top_p, frequency_penalty, presence_penalty, best_of, stop)
print(prompt_postfix)
response_text = print_response(response); print(response_text)

 
###
Here's what the above class is doing:
1.

The Message class is a dataclass that is frozen, meaning that it cannot be changed. It has two attributes, user and text. The render method returns a string with the user and text. 

2.
The Conversation class is a dataclass that is not frozen. It has one attribute, messages, which is a list of Message objects. The prepend method adds a message to the beginning of the list and the render method returns a string with all the messages separated by a separator token. 

3.
The Config class is a dataclass that is frozen. It has three attributes, name, instructions, and example_conversations. 

4.
The Prompt class is a dataclass that is frozen. It has four attributes, header, examples, and convo. The render method returns a string with the header, example conversations, and the current conversation.


In [22]:
output = prompt_postfix + response_text
print(output)

 
###
Here's what the above class is doing:
1.
The Message class is a dataclass that is frozen, meaning that it cannot be changed. It has two attributes, user and text. The render method returns a string with the user and text. 

2.
The Conversation class is a dataclass that is not frozen. It has one attribute, messages, which is a list of Message objects. The prepend method adds a message to the beginning of the list and the render method returns a string with all the messages separated by a separator token. 

3.
The Config class is a dataclass that is frozen. It has three attributes, name, instructions, and example_conversations. 

4.
The Prompt class is a dataclass that is frozen. It has four attributes, header, examples, and convo. The render method returns a string with the header, example conversations, and the current conversation.


### Save output to file

In [12]:
output_file_dir = "../output/explain_python_code/"
fname = os.path.join(output_file_dir, file_list[0] + '.output'); fname
output_file = open(fname, "w")
output_file.write(output)
output_file.close()

'../output/explain_python_code/base.py.output'

850